<a href="https://colab.research.google.com/github/josepmatias/ONE/blob/main/auto_one.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pip install pyxlsb

In [ ]:
#actualização de nomes de ficheiros e filtros

cadastro_nome="Cadastro_desbloqueado_18.6.xlsb"
servicos_nome="listagem activações com SDD.xlsb"
modelo_upfront_nome="Modelo upfront.xlsb"
listagem_cadastro="listagem cadastro maio 24.xlsb"
activacoes_neg="Ativações_Negativas_abr24_em_mai24_ONE.xlsb"

#actualização do nome de folhas
servicos_folha="Sheet1"
cadastro_folha="Cadastro de Ativos"

In [ ]:
#carregar ficheiros
from google.colab import files
uploaded = files.upload()
#uploaded2=files.upload()
#uploaded3=files.upload()
#uploaded4=files.upload()

In [ ]:
#função para retirar blanks e transformar em letras minúsculas
def formatar_palavra (palavra):
  return palavra.lower().strip()


In [ ]:
#ler ficheiro das activações - rever
activacoes_df=pd.read_excel(servicos_nome, sheet_name=servicos_folha)

activacoes_df.head(2)

print(f"a shape da listagem de activações é {activacoes_df.shape}")

#pegar na coluna "CONCATENATE" para criar coluna cav

#activacoes_df["cav"]=activacoes_df["CONCATENATE"].apply(lambda i:i[-len(i)+12:]).astype("str")

#formatar coluna "CAV" na listagem de activações
#activacoes_df["CAV"]=activacoes_df.loc[activacoes_df["cav"].isna()==False, "cav"].apply(lambda i: str(i).split(".")[0])

#criar coluna "user 2" na listagem de activações usando função "formatar_palavra"
#activacoes_df["user 2"]=activacoes_df["user"].apply(lambda i: formatar_palavra(i))
#print(f"o nº de linhas das activações do auto é {activacoes_df.iloc[:, 0].count()}")
#activacoes_df.head()

In [ ]:
#ler colunas a importar da listagem de activações

colunas=pd.read_excel("Modelo upfront (2).xlsb", sheet_name=1).iloc[:, 3].tolist()


activacoes_df=activacoes_df[colunas]

print(f"a shape da listagem de activações é {activacoes_df.shape}")

activacoes_df.head()

In [ ]:
#definir 1ª coluna da dataframe "activacoes_df" como "Activações"

activacoes_df["Tipo"]="Activações"

activacoes_df=activacoes_df[["Tipo"] + activacoes_df.iloc[:, :-2].columns.tolist()]

activacoes_df.head(2)

In [ ]:
#ler ficheiro cadastro
cadastro_df=pd.read_excel(cadastro_nome, sheet_name=cadastro_folha)

#criar coluna "User Aplicações 2" com letras minúsculas - criar nova coluna
cadastro_df["user aplicações 2"]=cadastro_df["User Aplicações"].apply(lambda i: formatar_palavra(i))

cadastro_df.head(2)

In [ ]:
#ler o modelo upfront

matriz_upfront=pd.read_excel(modelo_upfront_nome, sheet_name=0).iloc[:, [44, 45, 46]]

matriz_upfront.columns=matriz_upfront.iloc[0, :]

matriz_upfront.drop(0, inplace=True)

matriz_upfront.head()

In [ ]:
#ler a tabela com a matriz de pontuação

matriz_pontuacao=pd.read_excel(modelo_upfront_nome, sheet_name=0).iloc[:,51:56]

matriz_pontuacao.columns=matriz_pontuacao.iloc[0, :]

matriz_pontuacao=matriz_pontuacao.iloc[1:,:]

matriz_pontuacao2=matriz_pontuacao.dropna()

matriz_pontuacao2.head()

In [ ]:
#se análise daf for dth para cabo, CONTINUAR AQUI

#coluna "tipo_detalhe" com base na Analise DAF:
#. Comissionado Seguro -> Ciclo - Seguros
#. Comissionado Alarme -> Ciclo - Alarmes
#. Comissionada Navegação Segura -> Ciclo - Navegação Segura
#. Comissionado Router 6.0 | Comissionado Power wi-fi -> Ciclo - Equipamentos
#. restantes analise daf -> Ciclo - Serviços


In [ ]:
#criar coluna "Tipo_Detalhe"
analises=["Comissionado Seguro", "Comissionado Alarme", "Comissionada Navegação Segura", "Comissionado Router 6.0",\
"Comissionado Power wi-fi"]

detalhes=["Ciclo - Seguros", "Ciclo - Alarmes", "Ciclo - Navegação Segura", "Ciclo - Equipamentos", "Ciclo - Equipamentos"]

mascara2=pd.Series([True] * len(activacoes_df))
for i, j in zip(analises, detalhes):
  mascara=activacoes_df["Analise DAF"]==i
  mascara2|=mascara
  #print(activacoes_df.loc[mascara, "Analise DAF"].count())
  activacoes_df.loc[mascara, "Tipo_Detalhe"]=j

#preencher restantes linhas na coluna "Tipo_Detalhe" como "Ciclo - Serviços"

activacoes_df.loc[~activacoes_df["Analise DAF"].isin(analises), "Tipo_Detalhe"]="Ciclo - Serviços"

In [ ]:
#contabilização do nº de linhas na coluna "tipo detalhe"
activacoes_df["Tipo_Detalhe"].value_counts()

In [ ]:
#cálculo do valor de upfront

area=matriz_upfront[matriz_upfront.iloc[:, 0].isna()==False].iloc[:, 0].tolist()
upfront=matriz_upfront[matriz_upfront.iloc[:, 0].isna()==False].iloc[:, 1].tolist()

for i, j in zip(area, upfront):
  if activacoes_df.loc[activacoes_df["Sub-Área de Negócio"]==i, "Sub-Área de Negócio"].count()>0:
    activacoes_df.loc[activacoes_df["Sub-Área de Negócio"]==i, "Upfront € operador"]=j

#corrigir upfront de dth para cabo e pré-pagos, pois não são definidos com base na sub-area de negócio

mascaradth=activacoes_df["Analise DAF"].str.contains("dth para cabo", case=False)
mascarapre=activacoes_df["Analise DAF"].str.contains("pré-pago", case=False)

activacoes_df.loc[mascaradth, "Upfront € operador"]=5

activacoes_df.loc[mascarapre, "Upfront € operador"]=2.5


In [ ]:
#contabilização do nº de linhas na coluna "Upfront € operador"
activacoes_df["Upfront € operador"].value_counts()

In [ ]:
#se Analise DAF for "Comissionado Alarme", ver se há pontos extra consoante a "Campanha outbound (por CAV, Campanha Final)"

masc_alarme=activacoes_df["Analise DAF"].str.contains("alarme", case=False)

for i in activacoes_df.loc[masc_alarme, "Campanha outbound (por CAV, Campanha Final)"].tolist():
  pontos_adicionais=matriz_pontuacao2.loc[matriz_pontuacao2.iloc[:, 0]==i, :].iloc[:, -1].values[0].split(" ")[0]
  activacoes_df.loc[masc_alarme, "Ptos GCO Adicionais"]=pontos_adicionais



In [ ]:
#se Sub-Área de Negócio for "CABO", "DTH", "NET" e "VOIP" - criação da coluna "Ptos GCO Adicionais"
lista=["CABO", "DTH", "NET", "VOIP"]
activacoes_df.loc[activacoes_df["Sub-Área de Negócio"].isin(lista), "Campanha outbound (por CAV, Campanha Final)"]

def pont_extra (db, db2, col_out="Campanha outbound (por CAV, Campanha Final)", col="Sub-Área de Negócio", sub=["CABO", "DTH", "NET", "VOIP"]):
  #masc_sub -> filtro por cabo, dth, net, voip na sub-area
  masc_sub=db[col].isin(sub)
  #dataframe resumida com campanhas únicas
  db_short=pd.DataFrame(db.loc[masc_sub, col_out].unique(), columns=["campanha"])
  #merge com matriz_pontuacao2
  db_pont=db_short.merge(db2.loc[db2.iloc[:, 3].str.contains("2"), :], how="right", left_on="campanha", right_on="Campanha Final")
  lista2=db_pont.loc[db_pont["campanha"].isna()==False, "campanha"].unique()

  masc_camp=db[col_out].isin(lista2)
  #fixação pontos finais
  db.loc[masc_sub & masc_camp, "Ptos GCO Adicionais"]=2



pont_extra(activacoes_df, matriz_pontuacao2, "Campanha outbound (por CAV, Campanha Final)")

In [ ]:
#atribuição de nº de pontos - GCOpts
#se pagar for "sim"
#se Campanha outbound (por CAV, Campanha Final) é "up_alarmes_excl" -> pontos 0
#se Analise DAF é "Comissionada Navegação Segura", "Comissionado Seguro", "Comissionado Router 6.0", "Comissionado Power wi-fi" -> pontos 0
#se Analise DAF é "Comissionado Blitz BLM", procurar na matriz_pontuacao por "Prospect_blitz_blm"
#se Analise DAF é "Comissionado DTH para Cabo" procurar na matriz_pontuacao por "up_migracao_dth"
#os restantes procurar directamente na matriz_pontuacao

listazero=["Navegação", "Seguro", "Router 6.0", "wi-fi"] #análise daf

lista_alarmes=["up_alarmes_excl"]
dicionario={0:listazero+lista_alarmes, "prospect_blitz_blm": "Comissionado Blitz BLM", "up_migracao_dth": "Comissionado DTH para Cabo"}
listadaf=[]

def pontos_cat(db, db2, dici=dicionario, col1="Analise DAF", col2="Campanha outbound (por CAV, Campanha Final)"): #pts GCO
  #fixar campanhas a zero
  for i, j in dici.items():
    if i==0: #se j não for "up_alarmes_excl", vai procurar na análise daf
      for count in j:
        if count!="up_alarmes_excl":
          masc1=db[col1].str.contains(count, case=False)
          db.loc[masc1, "pts GCO"]=0
          listadaf.extend(db.loc[masc1, col1].tolist())
        else:
          masc1=db[col2].str.contains(count, case=False)
          db.loc[masc1, "pts GCO"]=0
    else: #up_migracao_dth e prospect_blitz_blm
      masc3=db[col1]==j #se for up_migracao_dth ou prospect_blitz_blm
      #print(j + " -> " + str(db.loc[masc3, col1].count()))
      valor=db2.loc[db2.iloc[:, 0]==i, :].iloc[:, 2].values[0]
      db.loc[masc3, "pts GCO"]=valor
      listadaf.extend(db.loc[masc3, col1].tolist())

pontos_cat(activacoes_df, matriz_pontuacao2)

In [ ]:
#obter a pontuação da coluna "pts GCO" com base na coluna campanha outbound e que não é incluído nas excepções anteriores

def pontuacao2 (db, db2, col_pt="pts GCO", col_camp="Campanha outbound (por CAV, Campanha Final)"):
  mask1=db[col_pt].isna()==True
  #campanhas=db.loc[mask1, col_camp]
  db.loc[mask1, col_pt]=db.loc[mask1, [col_camp]].merge(db2, how="left", left_on=col_camp, right_on="Campanha Final").iloc[:, 3].tolist()
pontuacao2(activacoes_df, matriz_pontuacao2)


In [ ]:
#somar colunas "pts GCO" e "Ptos GCO Adicionais"

activacoes_df.loc[activacoes_df["Ptos GCO Adicionais"].isna()==True, "Ptos GCO Adicionais"]=0

activacoes_df["pts GCO"]=\
list(map(lambda i, j: i+j, activacoes_df["Ptos GCO Adicionais"].astype("float").tolist(), activacoes_df["pts GCO"].astype("float").tolist()))

In [ ]:
#ler matriz info_autos

matriz_info=pd.read_excel(modelo_upfront_nome, sheet_name=0).iloc[:,33:42]

matriz_info.columns=matriz_info.iloc[0, :]

matriz_info=matriz_info.iloc[1:, :]

matriz_info.head()

In [ ]:
#ler ficheiro de activações negativas - 24/6

from google.colab import files
uploaded = files.upload()

In [ ]:
#activacoes_neg=pd.read_excel(activacoes_neg, sheet_name=0)

activacoes_neg=pd.read_excel("Ativações_Negativas_abr24_em_mai24_ONE.xlsb", sheet_name=0)

activacoes_neg.head()

In [ ]:
print(f"o nº de colunas na listagem de activações é {len(activacoes_df.columns.tolist())}")

print(f"o nº de colunas da listagem de activações negativas é {len(activacoes_neg2.columns.tolist())}")

list(filter(lambda i: i not in activacoes_neg2.columns.tolist(), activacoes_df.columns.tolist()))



In [ ]:
#colunas a copiar do ficheiro de activações negativas para o auto:

lista_neg=list(filter(lambda i: i not in activacoes_df.columns.tolist(), activacoes_neg.columns.tolist()))

lista_activacoes=list(filter(lambda i: i in activacoes_neg.columns.tolist(), activacoes_df.columns.tolist()))

print(lista_activacoes)

activacoes_neg2=activacoes_neg.drop(columns=lista_neg)

activacoes_neg2=activacoes_neg2[lista_activacoes]

activacoes_neg2.head(2)

#activacoes_df2.columns

print(len(activacoes_neg2.columns.tolist()))

In [ ]:
#concatenar listagem de activações com listagem de activações negativas

activacoes_df2=pd.concat([activacoes_df, activacoes_neg2], axis=0, ignore_index=True)

print(f"a shape da listagem de activações é {activacoes_df.shape}")

print(f"a shape da listagem de activações negativas é {activacoes_neg2.shape}")

print(f"a shape da listagem de activações + listagem de activações negativas é {activacoes_df2.shape}")

In [ ]:
#calcular o objectivo para cada operador

activacoes_df[["user 2", "Carga Horária", "pts GCO"]].groupby(["user 2", "Carga Horária"]) ["pts GCO"].sum().reset_index()

#[i for i in activacoes_df.columns.tolist() if str(i).startswith("pts")]



In [ ]:
#ler ficheiro da listagem de cadastro
listagem_cadastrodf=pd.read_excel(listagem_cadastro, sheet_name=0)
listagem_cadastrodf.head()


In [ ]:
df=cadastro_df.groupby("user aplicações 2") ["Campanha Final"].nunique().reset_index()
df[df["Campanha Final"].astype("int")>1]



In [ ]:
#criar tabela "cadastro_auto" para criar coluna auto no ciclo de activações

#certificar que cada cav outbound tem uma única "campanha final"
print("o nº de campanhas finais por cav são: ")
print(cadastro_df.groupby("user aplicações 2") ["Campanha Final"].nunique().unique())

cadastro_auto=cadastro_df[["user aplicações 2", "Campanha Final"]].drop_duplicates("user aplicações 2", ignore_index=True)

cadastro_auto.head()

In [ ]:
#criar coluna provisória "campanha final" para determinar o auto de cada cav

activacoes_df["campanha final"]=activacoes_df.merge(cadastro_auto, how="left", left_on="user 2", right_on="user aplicações 2").iloc[:, -1]

print(f"o nº de linhas é {activacoes_df.iloc[:, 0].count()}")

activacoes_df.head()

In [ ]:
activacoes_df.shape

In [ ]:
#determinar quantos users não têm campanha final

print(f"existem {activacoes_df.iloc[:, -1].isna().sum()} linhas sem campanha final determinada")
activacoes_df.loc[activacoes_df.iloc[:, -1].isna()==True, ["Tipo","user", "campanha final"]]


In [ ]:
#determinar a coluna auto com base na coluna "campanha final"

for i, j in zip(matriz_auto.iloc[:, 0], matriz_auto.iloc[:, 1]):
  activacoes_df.loc[activacoes_df["campanha final"]==i, "auto"]=j

print(f"existem {activacoes_df['auto'].isna().sum()} linhas sem auto determinado")

print(f"existem {activacoes_df.iloc[:, 0].count()} linhas no auto")

activacoes_df.auto.head()

In [ ]:
#determinar o supervisor para cada linha

#encontrar a coluna onde está o "supervisor"

[i for i in activacoes_df.columns.tolist() if str(i).find("supervisor")!=-1 or str(i).find("Supervisor")!=-1 or str(i).find("SUPERVISOR")!=-1]


#formatar coluna de supervisor na tabela de activações

activacoes_df["Supervisor/QT_GA_LIQ2"]=activacoes_df["Supervisor/QT_GA_LIQ2"].astype("str")

activacoes_df["Supervisor/QT_GA_LIQ2"]=activacoes_df["Supervisor/QT_GA_LIQ2"].apply(lambda j: j.split(".")[0])



#determinar username do supervisor para cada linha

activacoesdf_supervisor=\
pd.DataFrame(activacoes_df["Supervisor/QT_GA_LIQ2"].unique(), columns=["supervisor"])


#formatar coluna cav outbound no cadastro
cadastro_df["CAV Outbound"]=cadastro_df["CAV Outbound"].astype("str")

#fazer merge entre cavs de supervisores na listagem de activações e o cadastro

activacoesdf_supervisor=activacoesdf_supervisor.merge(cadastro_df[["CAV Outbound","user aplicações 2"]],\
how="left", left_on="supervisor", right_on="CAV Outbound")

#encontrar o username para o supervisor e actualização na listagem de activações

for i, j in zip(activacoesdf_supervisor.iloc[:, 0], activacoesdf_supervisor.iloc[:, 2]):
  activacoes_df.loc[activacoes_df["Supervisor/QT_GA_LIQ2"]==i, "username supervisor"]=j



In [ ]:
#encontrar username de supervisores com n/a

mask_sup1=activacoes_df["username supervisor"].isna()==True
mask_sup2=(activacoes_df["Supervisor/QT_GA_LIQ2"].isna()==True) | (activacoes_df["Supervisor/QT_GA_LIQ2"].str.isnumeric())

for i in activacoes_df.loc[mask_sup1 & mask_sup2, "Supervisor/QT_GA_LIQ2"].unique():
  user=listagem_cadastrodf.loc[listagem_cadastrodf.iloc[:, 1].astype("str")==i, "User Aplicações"].values[0]
  activacoes_df.loc[activacoes_df["Supervisor/QT_GA_LIQ2"]==i, "username supervisor"]=j


In [ ]:
#criar coluna "sub-area de negócio" no pressuposto de esta ainda não existir" - aplicável às linhas "ciclo serviços"
def sub_area_negocio(db, tipo, col_negocio, col_componente):
  if len([i for i in db.columns if i.startswith("Sub-")])==0:
    #db2=db[db["Tipo"]==tipo]
    for i in db[col_negocio].unique():
      if i=="BLM":
        db.loc[(db[col_negocio]==i) & (db["Tipo"]==tipo), "Sub-Área de Negócio"]="NET MÓVEL"
      elif i=="MOBILE":
        db.loc[(db[col_negocio]==i) & (db["Tipo"]==tipo), "Sub-Área de Negócio"]="PÓS-PAGO"
      elif i=="NET":
        db.loc[(db[col_negocio]==i) & (db["Tipo"]==tipo), "Sub-Área de Negócio"]="NET"
      elif i=="VOZ FIXA":
        db.loc[(db[col_negocio]==i) & (db["Tipo"]==tipo), "Sub-Área de Negócio"]="VOIP"
      elif i=="SEGUROS":
        db.loc[(db[col_negocio]==i) & (db["Tipo"]==tipo), "Sub-Área de Negócio"]="SEGUROS"
      elif i=="TV":
        for j in db[col_componente].unique():
          if j=="Max":
            db.loc[(db[col_componente]==j) & (db["Tipo"]==tipo), "Sub-Área de Negócio"]="DTH"
          else:
            db.loc[(db[col_componente]==j) & (db["Tipo"]==tipo), "Sub-Área de Negócio"]="CABO"
    return db["Sub-Área de Negócio"]

activacoes_df["Sub-Área de Negócio"]=sub_area_negocio(activacoes_df, "Ciclo - Serviços", "Negócio", "Componente")

activacoes_df.head()

In [ ]:
def sub_area_negocio_outros (db, col_tipo="Tipo"): #aplicável se não houver coluna "sub-área de negócio" para outros ciclos para além se "serviços"
  #if len([i for i in db.columns if i.startswith("Sub-")])==0:
  mask_tipo=(db[col_tipo].str.contains("ciclo", case=False)) & (~db[col_tipo].str.contains("serviço", case=False))
  lista=db.loc[mask_tipo, col_tipo].unique().tolist()
  print (lista)

  dici_ciclo={"equipamento":, "seguro":, "marcações": "-", "migrações":"-"}
sub_area_negocio_outros(activacoes_df)


In [ ]:
#definição da matriz de upfront operador
matriz_upfront=pd.read_excel(modelo_upfront_nome, sheet_name=0).iloc[:, 2:5]

matriz_upfront.columns=matriz_upfront.iloc[0, :]

matriz_upfront=matriz_upfront.drop(index=0)

matriz_upfront.dropna()

In [ ]:
#função para determinar valores upfront € no auto
def upfront (db, dbmatriz, mascara, colauto="Sub-Área de Negócio", colmatriz="Upfront"):

  for i, j in zip(dbmatriz.iloc[:, 0], dbmatriz.iloc[:, 1]):

    db.loc[mascara & (db[colauto]==i), "Upfront €"]=j

  return db


In [ ]:
#função para encontrar upfront €  nas migrações e marcações

def upfront_marcmigr (db, col_tipo, col_upfront="Upfront €"):
  #mask_tipo1=db[col_tipo].str.contains("marcações", case=False) |db[col_negocio].str.contains("migrações", case=False)
  mask_tipo1=db[col_tipo].str.contains("ciclo", case=False)

  for w, val in zip(["marcações", "migrações"], [1.5, 2.5]):
    db.loc[(mask_tipo1) & (db[col_tipo].str.contains(w, case=False)), col_upfront]=val

upfront_marcmigr(activacoes_df, "Tipo")

In [ ]:
activacoes_df.to_excel("output.xlsx", index=False)

In [ ]:
files.download("output.xlsx")

In [ ]:
#função para determinar os pontos no auto

def pontuacao (db, dbmatriz, mascara, colauto="Campanha outbound (por CAV, Campanha Final)", col_daf="Analise DAF", col_pontos="pts GCO"):
  #se análise daf for "Comissionada Navegação Segura", "Comissionado Seguro", "Comissionado Router 6.0" ou "Comissionado Power wi-fi", pontos é 0

  mask1=db[col_daf].isin(["Comissionada Navegação Segura", "Comissionado Seguro", "Comissionado Router 6.0", "Comissionado Power wi-fi"])

  db.loc[mask1, col_pontos]=0

  #se analise daf for "Comissionado Blitz BLM" ou "Comissionado DTH para Cabo" - vai procurar pelo valor do dicionario

  dici_extra={"Comissionado Blitz BLM": "prospect_blitz_blm", "Comissionado DTH para Cabo": "up_migracao_dth"}

  for i, j in dici_extra.items():
    val1=dbmatriz.loc[dbmatriz.iloc[:, 0]==j, col_pontos].values[0] #a coluna de pontos no auto e na matriz pontuação chama-se "pts GCO"
    db.loc[db[col_daf]==i, col_pontos]=val1

  #encontrar pontos para as restantes campanhas (normais)
  for i, j in zip(dbmatriz.iloc[:, 0], dbmatriz.iloc[:, 1]):
    db.loc[db[colauto]==i, col_pontos]=j

pontuacao(activacoes_df, matriz_pontuacao, mascara=lista_mascara)


In [ ]:
#definir máscaras para determinar upfront € para cada "tipo"
tipo_mask=activacoes_df["Tipo"].str.contains("serviço", case=False) & activacoes_df["Tipo"].str.contains("ciclo", case=False)
tipo_maskseguros=activacoes_df["Tipo"].str.contains("seguro", case=False) & activacoes_df["Tipo"].str.contains("ciclo", case=False)
tipos_maskmigracoes=activacoes_df["Tipo"].str.contains("migrações", case=False) & activacoes_df["Tipo"].str.contains("ciclo", case=False)
tipo_maskmarcacoes=activacoes_df["Tipo"].str.contains("marcações", case=False) & activacoes_df["Tipo"].str.contains("ciclo", case=False)

#máscara coluna negócio - excluir alarmes
negocio_mask1=activacoes_df["Negócio"].str.contains("alarme", case=False)

daf_mask1=activacoes_df["Analise DAF"].str.contains("dth para cabo", case=False)
daf_mask2=activacoes_df["Analise DAF"].str.contains("pré-pago", case=False)





In [ ]:
#calcular coluna "upfront €" para ciclo-serviços e sem dth para cabo nem pré-pagos
lista_mascara=tipo_mask & ~negocio_mask1 & ~daf_mask1 & ~daf_mask2

df_activacoes=upfront(activacoes_df, matriz_upfront, mascara=lista_mascara)

In [ ]:
#calcular coluna "upfront €" para ciclo-serviços e com dth para cabo

lista_mascara=tipo_mask & ~negocio_mask1 & daf_mask1 & ~daf_mask2
activacoes_df.loc[lista_mascara, "Upfront €"]=5
print(activacoes_df.loc[lista_mascara, "Upfront €"].count())

In [ ]:
#calcular coluna "upfront €" para ciclo-serviços e com pré-pagos

lista_mascara=tipo_mask & ~negocio_mask1 & ~daf_mask1 & daf_mask2

activacoes_df.loc[lista_mascara, "Upfront €"]=2.5
print(activacoes_df.loc[lista_mascara, "Upfront €"].count())

In [ ]:
activacoes_df.loc[tipo_mask, "Upfront €"].unique()

PARTE ANTIGA

In [ ]:
#actualização de nomes de ficheiros e filtros

activacao_negativa="Ativações_Negativas_fev24_em_mar24_ONE.xlsb"

cadastro="Cadastro.xlsb" #copia-se o cadastro original para um ficheiro à parte antes de importar

capitalizacao="02.2024 - Capitalização_Eq. Interna Obv.xlsb"

filtro_cap="Activação"

In [ ]:
#carregar e definir ficheiro de activações negativas

from google.colab import files
uploaded = files.upload()
uploaded2=files.upload()
uploaded3=files.upload()


activacao_negativa_df=pd.read_excel(activacao_negativa, sheet_name=0)
cadastro_df=pd.read_excel(cadastro, sheet_name=0)
capitalizacao_df=pd.read_excel(capitalizacao, sheet_name="act")

In [ ]:
#função para criar a coluna cav e adicioná-la à bd de activações negativas

extensao_total="40168941189470163540"

def extractor (str1):

  dicionario_extensao={"entidade": 6, "delegacao": (6,12), "cav": 12}

  for i, j in dicionario_extensao.items():
    if i=="delegacao":
      start_index, end_index=j
      delegacao=str1[start_index:end_index]
    elif i=="cav":
      start_index=j
      cav=str1[start_index:]
    else:
      entidade=str1[:j]
  return cav


activacao_negativa_df["cav"]=activacao_negativa_df["CONCATENATE"].apply(extractor).astype("int")



activacao_negativa_df["cav"].head()


In [ ]:
#eliminar duplicados no ficheiro do cadastro

cadastro_short=cadastro_df[["Cav outbound", "User Aplicações"]].drop_duplicates()


In [ ]:
#criar a coluna username na listagem de activações negativas


#converter cav outbound numa coluna "string"

#cadastro_short["Cav outbound"]=cadastro_short["Cav outbound"].astype("str")

activacao_negativa_df2= \
activacao_negativa_df.merge(cadastro_short[["Cav outbound", "User Aplicações"]], right_on="Cav outbound", left_on="cav", how="left")

activacao_negativa_df2.head()


print(f"existem {activacao_negativa_df2['User Aplicações'].isna().sum()} valores nulos na dataframe")

print(activacao_negativa_df2.shape)

activacao_negativa_df2.drop("Cav outbound", axis=1) #eliminar coluna "Cav outbound" que vem do cadastro

activacao_negativa_df2.loc[activacao_negativa_df2["cav"].isna()==True, ["cav", "Cav outbound"]]

In [ ]:
#filtrar capitalização por linhas de activação negativa

capitalizacao_df2=capitalizacao_df.loc[capitalizacao_df["Tipo.1"]==filtro_cap, ["Order Item", "valor unit pelos pontos Op", \
                    "valor unit pelos pontos Sup", "valor unit pelos pontos sup tr", \
                    "valor unit pelos pontos Co"]] #Tipo.1 é o nome da coluna que tem o tipo de movimento

#criar colunas que vêm da capitalização: quantidade op, quantidade sup, quantidade tc, quantidade coord, quantidade coord2

activacao_negativa_df3=activacao_negativa_df2.merge(capitalizacao_df2, on="Order Item", how="left")

activacao_negativa_df3.shape



In [ ]:
#multiplicar os valores das activações por -1 (porque são negativas)

activacao_negativa_df3=\
activacao_negativa_df3.loc[:, ["valor unit pelos pontos Op", "valor unit pelos pontos Sup", "valor unit pelos pontos sup tr", \
                           "valor unit pelos pontos Co"]].apply(lambda j: j*-1)

In [ ]:
activacao_negativa_df3.to_excel("output.xlsx", index=False)

In [ ]:
#fazer download do ficheiro

files.download('output.xlsx')